In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd



from IPython.display import display, Markdown

In [ ]:
# Parameters

input_path = "/data/amandeep/wikidata-20211027-dwd-v3"
output_path = "/data/amandeep/wikidata-20211027-dwd-v3"

graph_cache_path = None

project_name = "geography-files"

files = 'label,item,monolingualtext,external_id,p279star,p31'

In [ ]:
files = files.split(',')

In [ ]:
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                 graph_cache_path=graph_cache_path)

In [ ]:
ck.print_env_variables()

In [ ]:
ck.load_files_into_cache()

Mini node browser to help with debugging

# Build Files To Reason About Geography

Build a file that maps settlements to countries

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100020 | P17 | Q55 | Q100020-P17-Q55 |
| Q1000595 | P17 | Q43 | Q1000595-P17-Q43 |
| Q1000996 | P17 | Q183 | Q1000996-P17-Q183 |
| Q100141 | P17 | Q38 | Q100141-P17-Q38 |

# To Do: 

why did I put the where clause in the following query to filter out human settlement? the file becomes incomplete

In [9]:
!$kypher -i p31 -i items -i p279star \
--match ' \
    p31: (country)-[:P31]->(:Q6256), \
    items: (settlement)-[:P17]->(country), \
    p31: (settlement)-[:P31]->(a_settlement_subclass), \
    p279star: (a_settlement_subclass)-[]->(:Q486972)' \
--where 'a_settlement_subclass != "Q486972"' \
--return 'settlement as node1, "P17" as label, country as node2' \
/ compact -i - --deduplicate --build-id --id-style wikidata \
-o "$TEMP"/settlement-to-country.tsv.gz

[2021-08-08 18:26:38 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c2."node1" "_aLias.node1", ? "_aLias.label", graph_3_c1."node1" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_3 AS graph_3_c1, graph_3 AS graph_3_c3, graph_6 AS graph_6_c4
     ON graph_1_c2."node1" = graph_3_c3."node1"
        AND graph_3_c1."node1" = graph_1_c2."node2"
        AND graph_3_c3."node2" = graph_6_c4."node1"
        AND graph_1_c2."label" = ?
        AND graph_3_c1."label" = ?
        AND graph_3_c1."node2" = ?
        AND graph_3_c3."label" = ?
        AND graph_6_c4."node2" = ?
        AND (graph_3_c3."node2" != ?)
  PARAS: ['P17', 'P17', 'P31', 'Q6256', 'P31', 'Q486972', 'Q486972']
---------------------------------------------
     1899.78 real       194.99 user       238.53 sys


In [10]:
!$kypher -i "$TEMP"/settlement-to-country.tsv.gz --as settlements --limit 3

[2021-08-08 18:58:15 sqlstore]: DROP graph data table graph_25 from settlements
[2021-08-08 18:58:17 sqlstore]: IMPORT graph directly into table graph_25 from /Users/pedroszekely/Downloads/kypher/temp.geography/settlement-to-country.tsv.gz ...
[2021-08-08 18:58:23 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_25 AS graph_25_c1
     LIMIT ?
  PARAS: [3]
---------------------------------------------
node1	label	node2	id
Q100	P17	Q30	Q100-P17-Q30
Q100000	P17	Q55	Q100000-P17-Q55
Q1000003	P17	Q142	Q1000003-P17-Q142
        8.84 real        11.03 user         1.35 sys


## `derived.P17_chain`
a file so that chains any property whose value is a settlement to have a direct link to the country

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1000242 | P19_P17 | Q183 | Q1000242-P19_P17-Q183 |
| Q1000862 | P19_P17 | Q40 | Q1000862-P19_P17-Q40 |
| Q100142684 | P1427_P17 | Q869 | Q100142684-P1427_P17-Q869 |
| Q100152749 | P19_P17 | Q183 | Q100152749-P19_P17-Q183 |

In [11]:
!$kypher -i settlements -i items \
--match ' \
    items: (n1)-[l {label: property}]->(settlement), \
    settlements: (settlement)-[]->(country)' \
--return 'distinct n1 as node1, printf("%s_P17", property) as label, country as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.P17_chain.tsv.gz

[2021-08-08 18:58:27 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", printf(?, graph_1_c1."label") "_aLias.label", graph_25_c2."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_25 AS graph_25_c2
     ON graph_1_c1."node2" = graph_25_c2."node1"
        AND graph_1_c1."label" = graph_1_c1."label"
  PARAS: ['%s_P17']
---------------------------------------------
[2021-08-08 18:58:27 sqlstore]: CREATE INDEX on table graph_25 column node1 ...
[2021-08-08 18:58:28 sqlstore]: ANALYZE INDEX on table graph_25 column node1 ...
     1218.93 real       217.59 user       189.22 sys


# New files (Ameandeed: start here)

## `P131star`: map each node1 in `P131` to all its admin up the chain of `P131`

Removes historical admins, including historical countries.

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1000398 | P131star | Q12694 | Q1000398-P131star-Q12694 |
| Q1001008 | P131star | Q12589 | Q1001008-P131star-Q12589 |
| Q1001499 | P131star | Q214 | Q1001499-P131star-Q214 |
| Q1001995 | P131star | Q1001995 | Q1001995-P131star-Q1001995 |

Extract the graph of `P131` so that we give a smaller graph to reachable-nodes, and remove historical admins

In [106]:
!$kypher -i items -i p31 -i p279star \
--match '(n1)-[l:P131]->(n2)' \
--return 'distinct n1 as node1, l.label as label, n2 as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$TEMP"/P131.tsv.gz

[2021-09-19 22:26:26 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."label" "_aLias.label", graph_8_c1."node2" "_aLias.node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label" = ?
  PARAS: ['P131']
---------------------------------------------
      241.37 real       128.39 user        49.72 sys


Index P131

In [107]:
!$kypher -i "$TEMP"/P131.tsv.gz --as p131 --limit 2

[2021-09-19 22:30:26 sqlstore]: DROP graph data table graph_22 from p131
[2021-09-19 22:30:38 sqlstore]: IMPORT graph directly into table graph_26 from /Users/pedroszekely/Downloads/kypher/temp.geography/P131.tsv.gz ...
[2021-09-19 22:31:15 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_26 AS graph_26_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P2618	P131	Q3206	P2618-P131-Q3206
P2621	P131	Q21	P2621-P131-Q21
       50.38 real        63.64 user         8.42 sys


Get all node1 in the `P131` relation, as we will use them as roots for `P131star`

In [108]:
!$kypher -i p131 \
--match 'p131: (n1)-[]->(n2)' \
--return 'distinct n1 as id' \
-o "$TEMP"/p131.node1.tsv.gz

[2021-09-19 22:31:16 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_26_c1."node1" "_aLias.id"
     FROM graph_26 AS graph_26_c1
  PARAS: []
---------------------------------------------
       39.59 real        38.10 user         1.14 sys


Compute `P131star`, which maps every node1 in P131 to all the admin locations that can be reached from it.

In [109]:
!$kgtk reachable-nodes \
    --rootfile "$TEMP"/p131.node1.tsv.gz \
    --rootfilecolumn id \
    --label "P131star" \
    --selflink \
    -i "$TEMP"/P131.tsv.gz \
/ add-id --id-style wikidata \
/ sort \
-o $OUT/derived.P131star.tsv.gz

    41679.07 real     22700.02 user     19487.15 sys


Index `p131star` in kypher

In [110]:
!$kypher -i $OUT/derived.P131star.tsv.gz --as p131star --limit 2

[2021-09-20 10:06:36 sqlstore]: DROP graph data table graph_24 from p131star
[2021-09-20 10:07:40 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/geography/derived.P131star.tsv.gz ...
[2021-09-20 10:11:46 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P2618	P131star	P2618	P2618-P131star-P2618
P2618	P131star	Q3206	P2618-P131star-Q3206
      311.49 real       413.48 user        43.90 sys


Test: get all the admins of Pasadena: Q485176

In [111]:
if debug:
    !$kypher -i p131star -i labels -i p31 \
    --match ' \
        p131star: (:Q485176)-[]->(admin), \
        p31: (admin)-[]->(admin_class)' \
    --opt 'labels: (admin)-[]->(admin_label)' \
    --opt 'labels: (admin_class)-[]->(admin_class_label)' \
    --return 'distinct admin as admin, admin_label as admin_label, admin_class as admin_class, admin_class_label as admin_class_label' \
    --order-by 'admin_class_label' \
    | column -t -s $'\t'

[2021-09-20 10:11:47 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2" "_aLias.admin", graph_4_c3."node2" "_aLias.admin_label", graph_1_c2."node2" "_aLias.admin_class", graph_4_c4."node2" "_aLias.admin_class_label"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1
     ON graph_24_c1."node2" = graph_1_c2."node1"
        AND graph_24_c1."node1" = ?
     LEFT JOIN graph_4 AS graph_4_c3
     ON graph_24_c1."node2" = graph_4_c3."node1"
     LEFT JOIN graph_4 AS graph_4_c4
     ON graph_1_c2."node2" = graph_4_c4."node1"
     ORDER BY "_aLias.admin_class_label" ASC
  PARAS: ['Q485176']
---------------------------------------------
[2021-09-20 10:11:47 sqlstore]: CREATE INDEX on table graph_24 column node1 ...
[2021-09-20 10:12:13 sqlstore]: ANALYZE INDEX on table graph_24 column node1 ...
[2021-09-20 10:12:17 sqlstore]: CREATE INDEX on table graph_24 column node2 ...
[2021-09-20 10:13:11 sqlstore]: ANALYZE INDEX

##  `derived.P131admin2` map each human settlement to its admin2 `Q13220204`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100252368 | P131admin1 | Q1588 | Q100252368-P131admin1-Q1588 |
| Q1005394 | P131admin1 | Q54171 | Q1005394-P131admin1-Q54171 |
| Q100923 | P131admin1 | Q1263 | Q100923-P131admin1-Q1263 |
| Q101111580 | P131admin1 | Q34800 | Q101111580-P131admin1-Q34800 |

Make a list of all human settlements

In [84]:
!$kypher -i p31 -i p279star \
--match ' \
    p31: (human_settlement)-[]->(class), \
    p279star: (class)-[]->(:Q486972)' \
--return 'distinct human_settlement as node1' \
-o "$TEMP"/human_settlement.tsv.gz

[2021-09-19 21:21:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_6 AS graph_6_c2
     ON graph_1_c1."node2" = graph_6_c2."node1"
        AND graph_6_c2."node2" = ?
  PARAS: ['Q486972']
---------------------------------------------
       78.56 real        16.44 user        11.93 sys


Index human_settlement

In [85]:
!$kypher -i "$TEMP"/human_settlement.tsv.gz --as settlement --limit 2

[2021-09-19 21:22:38 sqlstore]: DROP graph data table graph_23 from settlement
[2021-09-19 21:22:39 sqlstore]: IMPORT graph directly into table graph_23 from /Users/pedroszekely/Downloads/kypher/temp.geography/human_settlement.tsv.gz ...
[2021-09-19 21:22:41 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_23 AS graph_23_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
Q104214562
Q11208543
        4.30 real         4.45 user         1.02 sys


Pick out all the admins that are below admin2 `Q13220204`

In [112]:
!$kypher -i p31 -i p131star -i p279star \
--match ' \
    p131star: (x)-[]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q13220204)' \
--return 'distinct x as node1, "P131admin2" as label, admin as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.P131admin2.tsv.gz

[2021-09-20 10:13:18 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node1" "_aLias.node1", ? "_aLias.label", graph_24_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1, graph_6 AS graph_6_c3
     ON graph_1_c2."node2" = graph_6_c3."node1"
        AND graph_24_c1."node2" = graph_1_c2."node1"
        AND graph_6_c3."node2" = ?
  PARAS: ['P131admin2', 'Q13220204']
---------------------------------------------
       71.88 real       100.86 user        13.61 sys


##  `derived.P131admin1` map each human settlement to its admin1 `Q10864048`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100252368 | P131admin1 | Q1588 | Q100252368-P131admin1-Q1588 |
| Q1005394 | P131admin1 | Q54171 | Q1005394-P131admin1-Q54171 |
| Q100923 | P131admin1 | Q1263 | Q100923-P131admin1-Q1263 |
| Q101111580 | P131admin1 | Q34800 | Q101111580-P131admin1-Q34800 |

Pick out the admins that are below admin1 `Q10864048`

In [113]:
!$kypher -i p31 -i p131star -i p279star \
--match ' \
    p131star: (x)-[]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q10864048)' \
--return 'distinct x as node1, "P131admin1" as label, admin as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.P131admin1.tsv.gz

[2021-09-20 10:14:30 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node1" "_aLias.node1", ? "_aLias.label", graph_24_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1, graph_6 AS graph_6_c3
     ON graph_1_c2."node2" = graph_6_c3."node1"
        AND graph_24_c1."node2" = graph_1_c2."node1"
        AND graph_6_c3."node2" = ?
  PARAS: ['P131admin1', 'Q10864048']
---------------------------------------------
       89.83 real       145.11 user        13.62 sys


## `derived.P131country` map each settlement to its country
Removes historical country

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003172 | P131country | Q45 | Q1003172-P131country-Q45 |
| Q100701578 | P131country | Q678 | Q100701578-P131country-Q678 |
| Q1010068 | P131country | Q219 | Q1010068-P131country-Q219 |
| Q101218885 | P131country | Q822 | Q101218885-P131country-Q822 |

Pick out all the admin that are countries `Q6256`

In [114]:
!$kypher -i p31 -i p131star -i p279star \
--match ' \
    p131star: (x)-[]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q6256)' \
--return 'distinct x as node1, "P131country" as label, admin as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.P131country.tsv.gz

[2021-09-20 10:16:00 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node1" "_aLias.node1", ? "_aLias.label", graph_24_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1, graph_6 AS graph_6_c3
     ON graph_1_c2."node2" = graph_6_c3."node1"
        AND graph_24_c1."node2" = graph_1_c2."node1"
        AND graph_6_c3."node2" = ?
  PARAS: ['P131country', 'Q6256']
---------------------------------------------
      154.81 real       220.65 user        17.97 sys


## `derived.P131.admin`

In [120]:
!ls "$OUT"/derived.P131*.tsv.gz

/Users/pedroszekely/Downloads/kypher/geography/derived.P131.admin.tsv.gz
/Users/pedroszekely/Downloads/kypher/geography/derived.P131admin1.tsv.gz
/Users/pedroszekely/Downloads/kypher/geography/derived.P131admin2.tsv.gz
/Users/pedroszekely/Downloads/kypher/geography/derived.P131country.tsv.gz
/Users/pedroszekely/Downloads/kypher/geography/derived.P131star.tsv.gz


In [121]:
!$kgtk cat \
-i "$OUT"/derived.P131admin1.tsv.gz \
-i "$OUT"/derived.P131admin2.tsv.gz \
-i "$OUT"/derived.P131country.tsv.gz \
-o "$OUT"/derived.P131.admin.tsv.gz

      378.64 real       377.09 user         0.92 sys


Index `derived.P131.admin`

In [122]:
!$kypher -i "$OUT"/derived.P131.admin.tsv.gz --as p131admin --limit 2

[2021-09-20 11:32:06 sqlstore]: DROP graph data table graph_25 from p131admin
[2021-09-20 11:32:11 sqlstore]: IMPORT graph directly into table graph_25 from /Users/pedroszekely/Downloads/kypher/geography/derived.P131.admin.tsv.gz ...
[2021-09-20 11:34:50 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_25 AS graph_25_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P2618	P131admin1	Q3206	P2618-P131admin1-Q3206
P2621	P131admin1	Q21	P2621-P131admin1-Q21
      164.65 real       263.65 user         9.96 sys


Test

In [123]:
!$kypher -i p131admin -i labels \
--match 'p131admin: (:Q485176)-[:P131country]->(country)'

[2021-09-20 11:34:51 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_25 AS graph_25_c1
     WHERE graph_25_c1."label" = ?
        AND graph_25_c1."node1" = ?
  PARAS: ['P131country', 'Q485176']
---------------------------------------------
[2021-09-20 11:34:51 sqlstore]: CREATE INDEX on table graph_25 column node1 ...
[2021-09-20 11:35:10 sqlstore]: ANALYZE INDEX on table graph_25 column node1 ...
[2021-09-20 11:35:13 sqlstore]: CREATE INDEX on table graph_25 column label ...
[2021-09-20 11:35:30 sqlstore]: ANALYZE INDEX on table graph_25 column label ...
node1	label	node2	id
Q485176	P131country	Q30	Q485176-P131country-Q30
       43.44 real        33.12 user         8.25 sys


Something to worry about: some countries have end times:

In [102]:
!$kypher -i p31 -i labels -i qualifiers \
--match ' \
    p31: (country)-[]->(:Q6256), \
    p31: (country)-[r]->(class), \
    qualifiers: (r)-[:P582]->(endtime), \
    labels: (country)-[]->(country_label), \
    labels: (class)-[]->(class_label)' \
--return 'country, country_label, class_label, endtime' \
--limit 20

[2021-09-19 22:10:14 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node1", graph_4_c4."node2", graph_4_c5."node2", graph_7_c3."node2"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_1 AS graph_1_c2, graph_4 AS graph_4_c4, graph_4 AS graph_4_c5, graph_7 AS graph_7_c3
     ON graph_1_c1."node1" = graph_1_c2."node1"
        AND graph_1_c1."node1" = graph_4_c4."node1"
        AND graph_1_c2."id" = graph_7_c3."node1"
        AND graph_1_c2."node2" = graph_4_c5."node1"
        AND graph_1_c1."node2" = ?
        AND graph_7_c3."label" = ?
     LIMIT ?
  PARAS: ['Q6256', 'P582', 20]
---------------------------------------------
node1	node2	node2	node2
Q1000	'Gabon'@en	'French colonial empire'@en	^1960-01-01T00:00:00Z/9
Q1029	'Mozambique'@en	'overseas province of Portugal'@en	^1975-06-24T00:00:00Z/11
Q1146786	'Señorío of Cuzcatlán'@en	'country'@en	^1528-01-01T00:00:00Z/9
Q1155700	'Rattanakosin Kingdom'@en	'country'@en	^1932-06-24T00:00:00Z/11


Import the file into the kypher index and give it the alias `admin1`

In [15]:
!$kypher -i "$OUT"/derived.settlement.P131admin1.tsv.gz --as settlementadmin1 --limit 3

[2021-08-08 19:36:34 sqlstore]: IMPORT graph directly into table graph_29 from /Users/pedroszekely/Downloads/kypher/geography/derived.settlement.P131admin1.tsv.gz ...
[2021-08-08 19:36:38 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_29 AS graph_29_c1
     LIMIT ?
  PARAS: [3]
---------------------------------------------
node1	label	node2	id
Q100	P131admin1	Q771	Q100-P131admin1-Q771
Q100000	P131admin1	Q1093	Q100000-P131admin1-Q1093
Q1000003	P131admin1	Q16987	Q1000003-P131admin1-Q16987
        5.72 real         5.92 user         0.36 sys


# Aliases for cities

### `derived.alias.settlement.admin1.full.tsv`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100349105 | alias | 'Colonnacce Roman villa, Lazio'@en | Q100349105-alias-44bd19 |
| Q1007434 | alias | 'Frangovo, Struga Municipality'@en | Q1007434-alias-1a7cbb |
| Q101071309 | alias | '50-76 Stryiska Street, Lviv, Lwów Voivodeship'@en | Q101071309-alias-6c36ec |
| Q1012440 | alias | 'Ranong, Ranong'@en | Q1012440-alias-2af390 |

In [25]:
!$kypher -i settlementadmin1 -i labels \
--match '\
    settlementadmin1: (settlement)-[]->(admin1), \
    labels: (settlement)-[]->(settlement_label), \
    labels: (admin1)-[]->(admin1_label)' \
--return 'distinct settlement as node1, "alias" as label, \
    kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), kgtk_lqstring_text(admin1_label))) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.alias.settlement.admin1.full.tsv.gz

[2021-08-08 19:42:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c2."node2"), kgtk_lqstring_text(graph_5_c3."node2"))) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_29 AS graph_29_c1
     INNER JOIN graph_5 AS graph_5_c2, graph_5 AS graph_5_c3
     ON graph_29_c1."node1" = graph_5_c2."node1"
        AND graph_29_c1."node2" = graph_5_c3."node1"
  PARAS: ['alias', '%s, %s', 'en', 'language_qualified_string']
---------------------------------------------
[2021-08-08 19:42:19 sqlstore]: CREATE INDEX on table graph_29 column node1 ...
[2021-08-08 19:42:20 sqlstore]: ANALYZE INDEX on table graph_29 column node1 ...
[2021-08-08 19:42:20 sqlstore]: CREATE INDEX on table graph_29 column node2 ...
[2021-08-08 19:42:21 sqlstore]: ANALYZE INDEX on table graph_29 column node2 ...
       72.8

### `derived.alias.settlement.admin1.short.tsv`
Use the  short name (`P1813`) of the admin as the label

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1012017 | alias | 'Plainfield, IL'@en | Q1012017-alias-28be4a |
| Q1020218 | alias | 'Princeton, C.-B.'@en | Q1020218-alias-f54afb |
| Q1029922 | alias | 'Camp Pendleton North, Cal.'@en | Q1029922-alias-3876ac |
| Q1052502 | alias | 'Cedar Grove, IN'@en | Q1052502-alias-b1d922 |

In [26]:
!$kypher -i settlementadmin1 -i monolingual -i labels \
--match '\
    settlementadmin1: (settlement)-[]->(admin1), \
    labels: (settlement)-[]->(settlement_label), \
    monolingual: (admin1)-[:P1813]->(admin1_short_label)' \
--return 'distinct settlement as node1, "alias" as label, \
    kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), kgtk_lqstring_text(admin1_short_label))) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.alias.settlement.admin1.short.tsv.gz

[2021-08-08 19:43:28 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c2."node2"), kgtk_lqstring_text(graph_27_c3."node2"))) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_27 AS graph_27_c3
     INNER JOIN graph_29 AS graph_29_c1, graph_5 AS graph_5_c2
     ON graph_29_c1."node1" = graph_5_c2."node1"
        AND graph_29_c1."node2" = graph_27_c3."node1"
        AND graph_27_c3."label" = ?
  PARAS: ['alias', '%s, %s', 'en', 'language_qualified_string', 'P1813']
---------------------------------------------
       15.60 real        23.66 user         3.00 sys


### `derived.alias.settlement.admin1.iso.tsv`

Use the iso code (`P300`) of the admin as the label, after removing the country part of the code

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003425 | alias | 'Fourneaux, P'@en | Q1003425-alias-f75f90 |
| Q1007427 | alias | 'La Lande-de-Fronsac, B'@en | Q1007427-alias-0eaf01 |
| Q101073 | alias | 'Foresto Sparso, 25'@en | Q101073-alias-b6bf2f |
| Q1012377 | alias | 'Fabryczna, LB'@en | Q1012377-alias-e30d31 |

In [27]:
!$kypher -i settlementadmin1 -i external_ids -i labels \
--match '\
    settlementadmin1: (settlement)-[]->(admin1), \
    labels: (settlement)-[]->(settlement_label), \
    external_ids: (admin1)-[:P300]->(iso_code)' \
--return 'distinct settlement as node1, "alias" as label, \
    kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), substr(kgtk_unstringify(iso_code), 4))) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.alias.settlement.admin1.iso.tsv.gz

[2021-08-08 19:43:45 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c2."node2"), substr(kgtk_unstringify(graph_7_c3."node2"), ?))) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_29 AS graph_29_c1
     INNER JOIN graph_5 AS graph_5_c2, graph_7 AS graph_7_c3
     ON graph_29_c1."node1" = graph_5_c2."node1"
        AND graph_29_c1."node2" = graph_7_c3."node1"
        AND graph_7_c3."label" = ?
  PARAS: ['alias', '%s, %s', 4, 'en', 'language_qualified_string', 'P300']
---------------------------------------------
       29.17 real        36.65 user         4.48 sys


### `derived.alias.city.country.tsv`

Produce "city, country" aliases for big cities

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1023481 | alias | 'Ocilla, United States of America'@en | Q1023481-alias-51eee8 |
| Q1160417 | alias | 'Lébény, Hungary'@en | Q1160417-alias-e1447f |
| Q1341 | alias | 'Tolyatti, Soviet Union'@en | Q1341-alias-e387a3 |
| Q151920 | alias | 'Tiberias, Mandatory Palestine'@en | Q151920-alias-0fb665 |

Produce city/country aliases for all cities (Q515) and all countries

In [28]:
# Q515: city
!$kypher -i p31 -i p279star -i items -i labels \
--match ' \
    p31: (city)-[]->(city_class), \
    p279star: (city_class)-[]->(:Q515), \
    items: (city)-[:P17]->(country), \
    labels: (city)-[]->(city_label), \
    labels: (country)-[]->(country_label)' \
--return 'distinct city as node1, "alias" as label, \
    kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(city_label), kgtk_lqstring_text(country_label), 4)) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.alias.city.country.tsv.gz

[2021-08-08 19:44:15 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c4."node2"), kgtk_lqstring_text(graph_5_c5."node2"), ?)) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_1 AS graph_1_c3
     INNER JOIN graph_3 AS graph_3_c1, graph_5 AS graph_5_c4, graph_5 AS graph_5_c5, graph_6 AS graph_6_c2
     ON graph_1_c3."node2" = graph_5_c5."node1"
        AND graph_3_c1."node1" = graph_1_c3."node1"
        AND graph_3_c1."node1" = graph_5_c4."node1"
        AND graph_3_c1."node2" = graph_6_c2."node1"
        AND graph_1_c3."label" = ?
        AND graph_6_c2."node2" = ?
  PARAS: ['alias', '%s, %s', 4, 'en', 'language_qualified_string', 'P17', 'Q515']
---------------------------------------------
       31.83 real         8.04 user         4.59 sys


### `derived.alias.city.us.tsv`

For the US, produce special files that use `USA` and `United States` in addition to the official name `United States of America`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100 | alias | 'Boston, United States'@en | Q100-alias-d454d3 |
| Q100 | alias | 'Boston, USA'@en | Q100-alias-fd19d4 |
| Q1000030 | alias | 'Orange City, USA'@en | Q1000030-alias-63cb74 |
| Q1000030 | alias | 'Orange City, United States'@en | Q1000030-alias-a00c53 |
| Q1000065 | alias | 'Neosho, USA'@en | Q1000065-alias-3631c0 |
| Q1000065 | alias | 'Neosho, United States'@en | Q1000065-alias-6be460 |

!zcat < "$OUT"/derived.alias.city.us.tsv.gz | head -7 | kgtk md

In [29]:
!$kypher -i p31 -i p279star -i items -i labels \
--match ' \
    p31: (city)-[]->(city_class), \
    p279star: (city_class)-[]->(:Q515), \
    items: (city)-[:P17]->(:Q30), \
    labels: (city)-[]->(city_label)' \
--return 'distinct city as node1, "alias" as label, \
    kgtk_stringify(printf("%s, USA", kgtk_lqstring_text(city_label))) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$TEMP"/derived.alias.city.USA.tsv.gz

[2021-08-08 19:44:46 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c4."node2"))) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_1 AS graph_1_c3
     INNER JOIN graph_3 AS graph_3_c1, graph_5 AS graph_5_c4, graph_6 AS graph_6_c2
     ON graph_3_c1."node1" = graph_1_c3."node1"
        AND graph_3_c1."node1" = graph_5_c4."node1"
        AND graph_3_c1."node2" = graph_6_c2."node1"
        AND graph_1_c3."label" = ?
        AND graph_1_c3."node2" = ?
        AND graph_6_c2."node2" = ?
  PARAS: ['alias', '%s, USA', 'en', 'language_qualified_string', 'P17', 'Q30', 'Q515']
---------------------------------------------
      229.66 real        23.15 user        33.65 sys


In [30]:
!$kypher -i p31 -i p279star -i items -i labels \
--match ' \
    p31: (city)-[]->(city_class), \
    p279star: (city_class)-[]->(:Q515), \
    items: (city)-[:P17]->(:Q30), \
    labels: (city)-[]->(city_label)' \
--return 'distinct city as node1, "alias" as label, \
    kgtk_stringify(printf("%s, United States", kgtk_lqstring_text(city_label))) as `node2;kgtk:text`, \
    "en" as `node2;kgtk:language`, \
    "language_qualified_string" as `node2;kgtk:data_type`' \
/ implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True \
/ add-id --id-style wikidata \
/ sort \
-o "$TEMP"/derived.alias.city.united_states.tsv.gz

[2021-08-08 19:48:37 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1" "_aLias.node1", ? "_aLias.label", kgtk_stringify(printf(?, kgtk_lqstring_text(graph_5_c4."node2"))) "_aLias.node2;kgtk:text", ? "_aLias.node2;kgtk:language", ? "_aLias.node2;kgtk:data_type"
     FROM graph_1 AS graph_1_c3
     INNER JOIN graph_3 AS graph_3_c1, graph_5 AS graph_5_c4, graph_6 AS graph_6_c2
     ON graph_3_c1."node1" = graph_1_c3."node1"
        AND graph_3_c1."node1" = graph_5_c4."node1"
        AND graph_3_c1."node2" = graph_6_c2."node1"
        AND graph_1_c3."label" = ?
        AND graph_1_c3."node2" = ?
        AND graph_6_c2."node2" = ?
  PARAS: ['alias', '%s, United States', 'en', 'language_qualified_string', 'P17', 'Q30', 'Q515']
---------------------------------------------
      227.90 real        23.58 user        34.32 sys


Combine the files for the US into one file, as it might be useful by itself

In [31]:
!$kgtk cat \
    -i "$TEMP"/derived.alias.city.USA.tsv.gz \
    -i "$TEMP"/derived.alias.city.united_states.tsv.gz \
/ sort \
-o "$OUT"/derived.alias.city.us.tsv.gz

        2.53 real         2.65 user         0.67 sys


### `derived.alias.settlement.tsv` 
Combined file of all aliases for cities

In [32]:
!$kgtk cat \
    -i "$OUT"/derived.alias.settlement.admin1.iso.tsv.gz \
    -i "$OUT"/derived.alias.settlement.admin1.short.tsv.gz \
    -i "$OUT"/derived.alias.settlement.admin1.full.tsv.gz \
    -i "$OUT"/derived.alias.city.country.tsv.gz \
    -i "$OUT"/derived.alias.city.us.tsv.gz \
/ compact -i - --deduplicate --build-id --id-style wikidata \
-o "$OUT"/derived.alias.settlement.tsv.gz

       34.96 real        40.74 user         1.17 sys


In [33]:
!$kgtk validate -i "$OUT"/derived.alias.settlement.tsv.gz


Data lines read: 1364775
Data lines passed: 1364775
       77.01 real        75.92 user         0.38 sys


# OLD CODE xxxxxxxxxxxxxxxx

## `derived.settlement.P131admin1`
map cities to their `admin1` 

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003343 | P131admin1 | Q18677875 | Q1003343-P131admin1-Q18677875 |
| Q1007140 | P131admin1 | Q18678082 | Q1007140-P131admin1-Q18678082 |
| Q1010414 | P131admin1 | Q326203 | Q1010414-P131admin1-Q326203 |
| Q1012284 | P131admin1 | Q1527 | Q1012284-P131admin1-Q1527 |

In [25]:
x = !zcat < "$OUT"/derived.P131admin.tsv.gz | head -5000 | kgtk cat --every-nth-record 1000 | kgtk md
display(Markdown("\n".join(x)))

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003343 | P131admin1 | Q18677875 | Q1003343-P131admin1-Q18677875 |
| Q1007140 | P131admin1 | Q18678082 | Q1007140-P131admin1-Q18678082 |
| Q1010414 | P131admin1 | Q326203 | Q1010414-P131admin1-Q326203 |
| Q1012284 | P131admin1 | Q1527 | Q1012284-P131admin1-Q1527 |

In [12]:
# Q10864048: first-level administrative country subdivision
!$kypher -i settlements -i items -i p31 -i p279star \
--match ' \
    settlements: (settlement)-[]->(), \
    items: (settlement)-[:P131]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q10864048)' \
--return 'distinct settlement as node1, "P131admin1" as label, admin as node2' \
-o "$TEMP"/derived.settlement.P131admin1.direct.tsv.gz

[2021-08-08 19:18:43 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_25_c1."node1" "_aLias.node1", ? "_aLias.label", graph_1_c2."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_25 AS graph_25_c1, graph_3 AS graph_3_c3, graph_6 AS graph_6_c4
     ON graph_1_c2."node2" = graph_3_c3."node1"
        AND graph_25_c1."node1" = graph_1_c2."node1"
        AND graph_3_c3."node2" = graph_6_c4."node1"
        AND graph_1_c2."label" = ?
        AND graph_6_c4."node2" = ?
  PARAS: ['P131admin1', 'P131', 'Q10864048']
---------------------------------------------
      215.13 real        11.99 user        27.80 sys


For some cities, such as Pasadena (Q485176), the value if `P131` is LA country, so need one more hop to get to the state

In [13]:
# Q10864048: first-level administrative country subdivision
!$kypher -i settlements -i items -i p31 -i p279star \
--match ' \
    settlements: (settlement)-[]->(), \
    items: (settlement)-[:P131]->(admin_a)-[:P131]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q10864048)' \
--return 'distinct settlement as node1, "P131admin1" as label, admin as node2' \
-o "$TEMP"/derived.settlement.P131admin1.2hop.tsv.gz

[2021-08-08 19:22:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_25_c1."node1" "_aLias.node1", ? "_aLias.label", graph_1_c3."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_25 AS graph_25_c1, graph_3 AS graph_3_c4, graph_6 AS graph_6_c5
     ON graph_1_c2."node2" = graph_1_c3."node1"
        AND graph_1_c3."node2" = graph_3_c4."node1"
        AND graph_25_c1."node1" = graph_1_c2."node1"
        AND graph_3_c4."node2" = graph_6_c5."node1"
        AND graph_1_c2."label" = ?
        AND graph_1_c3."label" = ?
        AND graph_6_c5."node2" = ?
  PARAS: ['P131admin1', 'P131', 'P131', 'Q10864048']
---------------------------------------------
      836.33 real        44.27 user        97.10 sys


Combine the two files into a single file

In [14]:
!$kgtk cat \
    -i "$TEMP"/derived.settlement.P131admin1.direct.tsv.gz \
    -i "$TEMP"/derived.settlement.P131admin1.2hop.tsv.gz \
/ compact -i - --deduplicate --build-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.settlement.P131admin1.tsv.gz

       18.29 real        19.32 user         1.19 sys


### `derived.settlement.P131admin2.tsv`

Map settlements to admin2

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1000239 | P131admin2 | Q12626 | Q1000239-P131admin2-Q12626 |
| Q1000679 | P131admin2 | Q1758432 | Q1000679-P131admin2-Q1758432 |
| Q1001015 | P131admin2 | Q702381 | Q1001015-P131admin2-Q702381 |
| Q1001458 | P131admin2 | Q12589 | Q1001458-P131admin2-Q12589 |

In [41]:
# Q13220204: second-level administrative country subdivision
!$kypher -i settlements -i items -i p31 -i p279star \
--match ' \
    settlements: (settlement)-[]->(), \
    items: (settlement)-[:P131]->(admin), \
    p31: (admin)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q13220204)' \
--return 'distinct settlement as node1, "P131admin2" as label, admin as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/derived.settlement.P131admin2.tsv.gz

[2021-08-08 21:07:18 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_25_c1."node1" "_aLias.node1", ? "_aLias.label", graph_1_c2."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_25 AS graph_25_c1, graph_3 AS graph_3_c3, graph_6 AS graph_6_c4
     ON graph_1_c2."node2" = graph_3_c3."node1"
        AND graph_25_c1."node1" = graph_1_c2."node1"
        AND graph_3_c3."node2" = graph_6_c4."node1"
        AND graph_1_c2."label" = ?
        AND graph_6_c4."node2" = ?
  PARAS: ['P131admin2', 'P131', 'Q13220204']
---------------------------------------------
      409.24 real        27.59 user        51.26 sys


In [48]:
!zcat < "$OUT"/derived.settlement.P131admin2.tsv.gz | head

node1	label	node2	id
Q100	P131admin2	Q54072	Q100-P131admin2-Q54072
Q100000	P131admin2	Q28109138	Q100000-P131admin2-Q28109138
Q100000	P131admin2	Q9766	Q100000-P131admin2-Q9766
Q1000003	P131admin2	Q12689	Q1000003-P131admin2-Q12689
Q1000003	P131admin2	Q702409	Q1000003-P131admin2-Q702409
Q1000008	P131admin2	Q12689	Q1000008-P131admin2-Q12689
Q1000008	P131admin2	Q700495	Q1000008-P131admin2-Q700495
Q1000009	P131admin2	Q12689	Q1000009-P131admin2-Q12689
Q1000009	P131admin2	Q700495	Q1000009-P131admin2-Q700495
zcat: error writing to output: Broken pipe


## `derived.admin2.admin1`

Map admin2 to admin1

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1001799 | P131admin1 | Q27566 | Q1001799-P131admin1-Q27566 |
| Q1003231 | P131admin1 | Q1112985 | Q1003231-P131admin1-Q1112985 |
| Q1003948 | P131admin1 | Q83274 | Q1003948-P131admin1-Q83274 |
| Q1004597 | P131admin1 | Q80550 | Q1004597-P131admin1-Q80550 |

In [ ]:
!wd u 

Create a file with all admin2's

In [19]:
# Q13220204: second-level administrative country subdivision
!$kypher -i p31 -i p279star \
--match ' \
    p31: (admin2)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q13220204)' \
--return 'distinct admin2 as node1' \
-o "$TEMP"/admin2.tsv.gz

[2021-08-08 19:41:51 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1" "_aLias.node1"
     FROM graph_3 AS graph_3_c1
     INNER JOIN graph_6 AS graph_6_c2
     ON graph_3_c1."node2" = graph_6_c2."node1"
        AND graph_6_c2."node2" = ?
  PARAS: ['Q13220204']
---------------------------------------------
       10.92 real         1.51 user         1.54 sys


In [20]:
!$kypher -i "$TEMP"/admin2.tsv.gz --as admin2 --limit 2

[2021-08-08 19:42:02 sqlstore]: DROP graph data table graph_28 from admin2
[2021-08-08 19:42:03 sqlstore]: IMPORT graph directly into table graph_28 from /Users/pedroszekely/Downloads/kypher/temp.geography/admin2.tsv.gz ...
[2021-08-08 19:42:03 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_28 AS graph_28_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
Q1026448
Q1120443
        1.49 real         0.86 user         0.25 sys


Create a file with all admin1's

In [34]:
# Q10864048: first-level administrative country subdivision
!$kypher -i p31 -i p279star \
--match ' \
    p31: (admin1)-[]->(admin_class), \
    p279star: (admin_class)-[]->(:Q10864048)' \
--return 'distinct admin1 as node1' \
-o "$TEMP"/admin1.tsv.gz

[2021-08-08 20:57:04 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1" "_aLias.node1"
     FROM graph_3 AS graph_3_c1
     INNER JOIN graph_6 AS graph_6_c2
     ON graph_3_c1."node2" = graph_6_c2."node1"
        AND graph_6_c2."node2" = ?
  PARAS: ['Q10864048']
---------------------------------------------
        3.34 real         0.91 user         0.48 sys


In [35]:
!$kypher -i "$TEMP"/admin1.tsv.gz --as admin1 --limit 2

[2021-08-08 20:57:29 sqlstore]: DROP graph data table graph_26 from admin1
[2021-08-08 20:57:30 sqlstore]: IMPORT graph directly into table graph_26 from /Users/pedroszekely/Downloads/kypher/temp.geography/admin1.tsv.gz ...
[2021-08-08 20:57:30 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_26 AS graph_26_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
Q182378
Q1865
        1.79 real         0.87 user         0.68 sys


In [50]:
!$kypher -i admin1 -i admin2 -i items -i p31 -i p279star \
--match ' \
    admin1: (admin1)-[]->(), \
    admin2: (admin2)-[]->(), \
    items: (admin2)-[:P131]->(admin1)' \
--return 'distinct admin2 as node1, "P131admin1" as label, admin1 as node2' \
/ add-id --id-style wikidata \
-o "$TEMP"/admin2.P131admin1.admin1.tsv.gz

[2021-08-08 21:32:08 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_28_c2."node1" "_aLias.node1", ? "_aLias.label", graph_26_c1."node1" "_aLias.node2"
     FROM graph_1 AS graph_1_c3
     INNER JOIN graph_26 AS graph_26_c1, graph_28 AS graph_28_c2
     ON graph_26_c1."node1" = graph_1_c3."node2"
        AND graph_28_c2."node1" = graph_1_c3."node1"
        AND graph_1_c3."label" = ?
  PARAS: ['P131admin1', 'P131']
---------------------------------------------
      294.67 real        15.38 user        40.70 sys


Surprisingly, traversing `P131` twice may lead to other links

In [51]:
!$kypher -i admin1 -i admin2 -i items -i p31 -i p279star \
--match ' \
    admin1: (admin1)-[]->(), \
    admin2: (admin2)-[]->(), \
    items: (admin2)-[:P131]->()-[:P131]->(admin1)' \
--return 'distinct admin2 as node1, "P131admin1" as label, admin1 as node2' \
/ add-id --id-style wikidata \
-o "$TEMP"/admin2.P131admin1.2hop.admin1.tsv.gz

[2021-08-08 21:37:04 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_28_c2."node1" "_aLias.node1", ? "_aLias.label", graph_26_c1."node1" "_aLias.node2"
     FROM graph_1 AS graph_1_c3
     INNER JOIN graph_1 AS graph_1_c4, graph_26 AS graph_26_c1, graph_28 AS graph_28_c2
     ON graph_1_c3."node2" = graph_1_c4."node1"
        AND graph_26_c1."node1" = graph_1_c4."node2"
        AND graph_28_c2."node1" = graph_1_c3."node1"
        AND graph_1_c3."label" = ?
        AND graph_1_c4."label" = ?
  PARAS: ['P131admin1', 'P131', 'P131']
---------------------------------------------
       30.16 real         4.54 user         4.09 sys


Use `P50` to possibly find new links

In [52]:
# development query
# P150: contains administrative territorial entity
!$kypher -i admin1 -i admin2 -i items -i p31 -i p279star \
--match ' \
    admin1: (admin1)-[]->(), \
    items: (admin1)-[:P150]->(admin2), \
    admin2: (admin2)-[]->()' \
--return 'distinct admin2 as node1, "P131admin1" as label, admin1 as node2' \
/ add-id --id-style wikidata \
-o "$TEMP"/P150.admin2.P131admin1.admin1.tsv.gz

[2021-08-08 21:37:33 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c2."node2" "_aLias.node1", ? "_aLias.label", graph_26_c1."node1" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_26 AS graph_26_c1, graph_28 AS graph_28_c3
     ON graph_1_c2."node2" = graph_28_c3."node1"
        AND graph_26_c1."node1" = graph_1_c2."node1"
        AND graph_1_c2."label" = ?
  PARAS: ['P131admin1', 'P150']
---------------------------------------------
        3.92 real         2.96 user         0.75 sys


Concatenate all the `P131admin1` files

In [53]:
!kgtk cat \
    -i "$TEMP"/admin2.P131admin1.admin1.tsv.gz \
    -i "$TEMP"/admin2.P131admin1.admin1.tsv.gz \
    -i "$TEMP"/P150.admin2.P131admin1.admin1.tsv.gz \
/ sort \
-o "$OUT"/derived.admin2.admin1.tsv.gz

# END OLD CODE